# Content-based Filtering

Aproach:
- Table with features erstellen, ratings in zweitem table

- sklearn-kNN mit minkowski-distance auf features anwenden, erhalte matrix mit allen Nachbarn eines jeden Films inklusive Abstand

- Wähle für jeden Nutzer und Film die x-nächsten Nachbarn aus, die der Nutzer bewertet hat

- Funktion schreiben, die auf basis des ratings der neighbours das rating eines Filmes vorhersagt.

In [1]:
import pandas as pd
import numpy as np
import string
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
kwargs = dict(random_state=42)

In [2]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [3]:
movies = pd.read_csv('../../data/preprocessed/movies_id_updated.csv')
average_ratings = pd.read_csv('../../data/preprocessed/average_ratings.csv')
omdb = pd.read_csv('../../data/preprocessed/omdb_cleaned.csv')
ratings = pd.read_csv('../../data/preprocessed/ratings_clean_std_0.csv')

In [4]:
movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtPictureURL
0,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,tt0113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995.0,1068044-jumanji,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,tt0107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993.0,grumpy_old_men,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,tt0114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995.0,waiting_to_exhale,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,tt0113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995.0,father_of_the_bride_part_ii,http://content8.flixster.com/movie/25/54/25542...


In [5]:
ratings.head()

,Unnamed: 0,user_id,imdbID,rating
0,0,1264,tt0047034,3.5
1,1,213,tt0304141,2.5
2,2,593,tt0369436,3.0
3,3,609,tt1077258,4.0
4,4,1590,tt0052182,4.0


In [6]:
omdb.head()

,Title,Year,Rated,Runtime,Writer,Plot,Language,imdbRating,imdbVotes,imdbID,...,Series,Released_season,Released_month,Released_day,PG_Rating,Available_languages,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,Toy Story,1995.0,G,81.0,"John Lasseter (original story by), Pete Docter...",A cowboy doll is profoundly threatened and jea...,['English'],8.3,852896.0,tt0114709,...,0,4.0,11.0,22.0,0.0,1,0.0,3.0,0.0,0.0
1,Jumanji,1995.0,PG,104.0,"Jonathan Hensleigh (screenplay by), Greg Taylo...",When two kids find and play a magical board ga...,"['English', ' French']",7.0,294340.0,tt0113497,...,0,1.0,12.0,15.0,1.0,2,0.0,0.0,0.0,0.0
2,Grumpy Old Men,1993.0,PG-13,103.0,Mark Steven Johnson,A lifelong feud between two neighbors since ch...,['English'],7.0,41401.0,tt0107050,...,0,1.0,12.0,25.0,2.0,1,0.0,0.0,0.0,0.0
3,Waiting to Exhale,1995.0,R,124.0,"Terry McMillan (novel), Terry McMillan (screen...","Based on Terry McMillan's novel, this film fol...",['English'],5.9,9222.0,tt0114885,...,0,1.0,12.0,22.0,3.0,1,0.0,0.0,0.0,0.0
4,Father of the Bride Part II,1995.0,PG,106.0,"Albert Hackett (screenplay ""Father's Little Di...",George Banks must deal not only with the pregn...,['English'],6.0,33005.0,tt0113041,...,0,1.0,12.0,8.0,1.0,1,0.0,0.0,0.0,1.0


In [7]:
omdb.columns

Index(['Title', 'Year', 'Rated', 'Runtime', 'Writer', 'Plot', 'Language',
       'imdbRating', 'imdbVotes', 'imdbID', 'Rotten Tomatoes', 'Metacritic',
       'Series', 'Released_season', 'Released_month', 'Released_day',
       'PG_Rating', 'Available_languages', 'Oscars_won', 'Oscars_nominated',
       'Golden_globe_won', 'Golden_globe_nominated'],
      dtype='object')

In [8]:
# Select relevant columns
omdb = omdb[['imdbID', 'Year', 'Runtime', 'Language', 'imdbRating', 'imdbVotes', 'Rotten Tomatoes', 'Metacritic',
       'Series', 'Released_month', 'PG_Rating', 'Available_languages', 'Oscars_won', 'Oscars_nominated',
       'Golden_globe_won', 'Golden_globe_nominated']]

In [9]:
omdb.isna().sum()

imdbID                       0
Year                         0
Runtime                      9
Language                     0
imdbRating                   4
imdbVotes                    5
Rotten Tomatoes           1237
Metacritic                4019
Series                       0
Released_month              56
PG_Rating                 2314
Available_languages          0
Oscars_won                   0
Oscars_nominated             0
Golden_globe_won             0
Golden_globe_nominated       0
dtype: int64

In [10]:
# Add movies which could not be retrieved by omdb
imdb_ids = pd.DataFrame(movies['imdbID'].unique()).rename(columns={0:'imdbID'})
omdb = imdb_ids.merge(omdb, how='left', on='imdbID')

In [11]:
# missing entries in omdb dataset
omdb[omdb['Year'].isna()]['imdbID']

707     tt1233247
2111    tt0800175
2241    tt0134948
3399    tt0257001
3810    tt0266860
4822    tt0720339
5118    tt0313487
7581    tt0439663
8140    tt0383974
9411    tt0182989
Name: imdbID, dtype: object

In [12]:
omdb.isna().sum()

imdbID                       0
Year                        10
Runtime                     15
Language                    10
imdbRating                  10
imdbVotes                   11
Rotten Tomatoes           1246
Metacritic                4025
Series                      10
Released_month              62
PG_Rating                 2320
Available_languages         10
Oscars_won                  10
Oscars_nominated            10
Golden_globe_won            10
Golden_globe_nominated      10
dtype: int64

In [13]:
# Replace NaN with 0 and handle accordingly for awards
for i in range (9,13):
    omdb.iloc[:,i] = omdb.iloc[:,i].fillna(0)

In [14]:
omdb.loc[:,'Series'] = omdb.loc[:,'Series'].fillna(0)


In [15]:
# Comment Max: Vorschlag - Stattdessen Median/Modalwert benutzen benutzen
# Fill NaN Series
omdb.loc[:,'Series'] = omdb.loc[:,'Series'].fillna(0)
# Fill NaN Runtime
omdb.loc[:,'Runtime'] = omdb.loc[:,'Runtime'].fillna(omdb.loc[:,'Runtime'].median())
# Fill median for all others
for i in range (4,16):
    omdb.iloc[:,i] = omdb.iloc[:,i].fillna(omdb.iloc[:,i].median())

In [16]:
# possibility to encode language as one-hot encoding
#mlb = MultiLabelBinarizer()
#mlb.fit_transform(merged_data['Language'])

In [17]:
# convert imdbID from string to float
omdb['imdbID'] = omdb['imdbID'].str.replace(r'tt','')
omdb['imdbID'] = omdb['imdbID'].astype(float)
omdb['imdbID'].unique()
ratings['imdbID'] = ratings['imdbID'].str.replace(r'tt','')
ratings['imdbID'] = ratings['imdbID'].astype(float)

In [18]:
# delete language for the moment as not yet encoded
omdb = omdb.drop(columns={'Language'})

In [19]:
# fill NaN of the Year
omdb['Year'] = omdb['Year'].fillna(omdb['Year'].median())

In [20]:
# show first entry
omdb.loc[omdb['imdbID'] == 114709]

,imdbID,Year,Runtime,imdbRating,imdbVotes,Rotten Tomatoes,Metacritic,Series,Released_month,PG_Rating,Available_languages,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,114709.0,1995.0,81.0,8.3,852896.0,10.0,9.5,0.0,11.0,0.0,1.0,0.0,3.0,0.0,0.0


In [21]:
omdb.sort_values('imdbID').iloc[62,:]

imdbID                    20530.0
Year                       1929.0
Runtime                      16.0
imdbRating                    7.7
imdbVotes                 43871.0
Rotten Tomatoes              10.0
Metacritic                    5.9
Series                        0.0
Released_month                6.0
PG_Rating                     0.0
Available_languages           1.0
Oscars_won                    0.0
Oscars_nominated              0.0
Golden_globe_won              0.0
Golden_globe_nominated        0.0
Name: 7536, dtype: float64

In [22]:
# set index as imdbID in ascending order
#features = omdb.set_index('imdbID', drop = True)
#features = features.sort_index()

In [23]:
features = omdb

In [24]:
# Standardize the features
scaler = preprocessing.StandardScaler()
features = scaler.fit_transform(features)

In [25]:
features

array([[-0.40529431,  0.37886077, -0.99135219, ...,  3.68058431,
        -0.06819758, -0.16894619],
       [-0.41125947,  0.37886077, -0.04121391, ..., -0.27359947,
        -0.06819758, -0.16894619],
       [-0.44298994,  0.27105774, -0.08252427, ..., -0.27359947,
        -0.06819758, -0.16894619],
       ...,
       [-0.84453528, -2.90913151, -0.49562787, ..., -0.27359947,
        -0.06819758, -0.16894619],
       [ 4.0735215 ,  1.07958043, -0.53693823, ..., -0.27359947,
        -0.06819758, -0.16894619],
       [ 0.07639669,  0.59446682, -2.97424948, ..., -0.27359947,
        -0.06819758, -0.16894619]])

In [26]:
# features of the first entry (imdbID 439)
features[0]

array([-0.40529431,  0.37886077, -0.99135219,  1.68821913,  6.82544447,
        1.4084073 ,  2.62822861, -0.0807199 ,  1.26788359, -1.33063277,
       -0.52609534, -0.20232346,  3.68058431, -0.06819758, -0.16894619])

In [27]:
features.shape[0]

9423

In [28]:
# create NearesNeighbor model with the minkowski distance
knn = NearestNeighbors(metric='minkowski', p=2, algorithm='brute', n_jobs=-1)

In [29]:
# fit to all the features
knn.fit(features)

NearestNeighbors(algorithm='brute', n_jobs=-1)

In [30]:
# create two array for distance and index to store
distances = np.empty((features.shape[0],features.shape[0]))
indices = np.empty((features.shape[0],features.shape[0]))

# iterate over all imdbIDs
for i in range(0, features.shape[0]):
    # get all neighbors and the distances for each imdbID i
    distances[i], indices[i] = knn.kneighbors([features[i]], n_neighbors=features.shape[0])
    
    if i%1000==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [31]:
distances

array([[0.00000000e+00, 1.96588178e+00, 3.43476464e+00, ...,
        2.76537266e+01, 3.00939977e+01, 5.11368348e+01],
       [5.96046448e-08, 1.38212405e+00, 1.50351137e+00, ...,
        2.60586696e+01, 2.90000512e+01, 5.01867989e+01],
       [0.00000000e+00, 5.99194841e-01, 6.26245488e-01, ...,
        2.62639064e+01, 2.87775021e+01, 5.01684724e+01],
       ...,
       [0.00000000e+00, 4.73830252e-01, 5.57099486e-01, ...,
        2.70022490e+01, 2.90429548e+01, 5.05102478e+01],
       [5.96046448e-08, 8.40906513e-01, 9.05067368e-01, ...,
        2.69628453e+01, 2.85531815e+01, 5.03836549e+01],
       [0.00000000e+00, 7.40186730e-01, 9.45900558e-01, ...,
        2.86539889e+01, 2.89338556e+01, 5.08908690e+01]])

In [32]:
indices

array([[0.000e+00, 1.496e+03, 1.085e+03, ..., 3.796e+03, 9.019e+03,
        8.322e+03],
       [1.000e+00, 2.146e+03, 2.200e+02, ..., 3.796e+03, 9.019e+03,
        8.322e+03],
       [2.000e+00, 5.500e+01, 2.445e+03, ..., 3.796e+03, 9.019e+03,
        8.322e+03],
       ...,
       [9.420e+03, 7.264e+03, 4.721e+03, ..., 3.796e+03, 9.019e+03,
        8.322e+03],
       [9.421e+03, 9.328e+03, 9.362e+03, ..., 3.796e+03, 9.019e+03,
        8.322e+03],
       [9.422e+03, 6.400e+02, 7.145e+03, ..., 3.796e+03, 9.019e+03,
        8.322e+03]])

In [33]:
index = omdb.loc[(omdb['imdbID'] == 47034.0)].index
print(index)   

Int64Index([2103], dtype='int64')


In [138]:
distances_df = pd.DataFrame(distances)
distances_df

,0,1,2,3,4,5,6,7,8,9,...,9413,9414,9415,9416,9417,9418,9419,9420,9421,9422
0,0.000000e+00,1.965882,3.434765,3.833063,4.014939,4.190385,4.223806,4.272378,4.299922,4.354121,...,21.111036,21.197016,22.837028,23.637733,24.050727,26.703494,27.426812,27.653727,30.093998,51.136835
1,5.960464e-08,1.382124,1.503511,1.600917,1.824605,1.974457,2.003216,2.013830,2.026301,2.085728,...,19.417942,19.494496,20.834504,22.998420,23.069728,24.636283,25.827872,26.058670,29.000051,50.186799
2,0.000000e+00,0.599195,0.626245,0.660796,0.713484,0.759627,0.786329,0.837231,0.899750,0.907823,...,19.342515,20.905383,21.055273,23.085200,23.095087,24.530268,25.666862,26.263906,28.777502,50.168472
3,0.000000e+00,0.215642,0.414489,0.607333,0.631640,0.702686,0.753400,0.789306,0.800508,0.815224,...,19.508479,20.261391,21.354187,23.278595,23.350125,23.872552,25.393367,25.711595,28.814927,50.013952
4,5.960464e-08,0.827862,1.354993,1.358201,1.372418,1.474608,1.538451,1.538567,1.615121,1.616602,...,19.864035,21.310085,21.708098,23.513846,23.572297,24.730534,26.026715,26.652482,29.173908,50.370137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9418,0.000000e+00,1.021744,1.086956,1.217040,1.262637,1.309347,1.325024,1.328577,1.410411,1.444199,...,19.709021,21.251593,21.732177,23.092151,23.232685,24.941817,26.176752,26.817629,28.661248,50.335935
9419,5.960464e-08,1.097860,1.631347,1.656113,1.720468,1.752052,1.774851,1.781652,1.814852,1.825412,...,20.073727,21.628659,21.978164,23.327246,23.364031,24.936789,26.319034,26.899387,28.863882,50.412855
9420,0.000000e+00,0.473830,0.557099,0.680101,0.732550,0.743268,0.762679,0.808532,0.823190,0.844499,...,19.912109,21.356931,21.705791,23.317897,23.529214,25.004023,25.919261,27.002249,29.042955,50.510248
9421,5.960464e-08,0.840907,0.905067,0.983991,1.109585,1.181246,1.229947,1.261676,1.268800,1.311302,...,19.778646,21.765176,22.060392,23.156147,23.289271,25.131708,26.284827,26.962845,28.553182,50.383655


In [161]:
indices_df = pd.DataFrame(indices)
indices_df

,0,1,2,3,4,5,6,7,8,9,...,9413,9414,9415,9416,9417,9418,9419,9420,9421,9422
0,0.0,1496.0,1085.0,5402.0,4829.0,3795.0,1091.0,1100.0,721.0,2179.0,...,7775.0,7799.0,6211.0,1907.0,3481.0,6162.0,7739.0,3796.0,9019.0,8322.0
1,1.0,2146.0,220.0,1251.0,2647.0,7926.0,5029.0,2148.0,18.0,4179.0,...,306.0,7799.0,6211.0,1907.0,3481.0,6162.0,7739.0,3796.0,9019.0,8322.0
2,2.0,55.0,2445.0,6683.0,6302.0,1647.0,7105.0,4575.0,2814.0,4114.0,...,7799.0,6211.0,306.0,3481.0,1907.0,6162.0,7739.0,3796.0,9019.0,8322.0
3,3.0,2169.0,214.0,8.0,432.0,5772.0,4241.0,4052.0,341.0,7951.0,...,9101.0,6211.0,306.0,3481.0,1907.0,6162.0,7739.0,3796.0,9019.0,8322.0
4,4.0,4105.0,301.0,2735.0,3324.0,1914.0,3774.0,2166.0,2530.0,3611.0,...,9101.0,6211.0,306.0,3481.0,1907.0,6162.0,7739.0,3796.0,9019.0,8322.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9418,9418.0,9072.0,9153.0,9225.0,9029.0,9341.0,8868.0,8715.0,9219.0,9390.0,...,7799.0,6211.0,306.0,3481.0,1907.0,6162.0,7739.0,3796.0,9019.0,8322.0
9419,9419.0,9188.0,8660.0,9250.0,9296.0,8907.0,9031.0,9279.0,9380.0,8964.0,...,7799.0,306.0,6211.0,3481.0,1907.0,6162.0,7739.0,3796.0,9019.0,8322.0
9420,9420.0,7264.0,4721.0,6713.0,873.0,3259.0,875.0,5764.0,7263.0,8551.0,...,9101.0,6211.0,306.0,3481.0,1907.0,6162.0,7739.0,3796.0,9019.0,8322.0
9421,9421.0,9328.0,9362.0,9289.0,9361.0,9257.0,9300.0,4617.0,9279.0,440.0,...,7799.0,306.0,6211.0,3481.0,1907.0,6162.0,7739.0,3796.0,9019.0,8322.0


In [34]:
 index = omdb.loc[(omdb['imdbID'] == 47034.0)].index

In [162]:
movie_row = indices_df.loc[index]
movie_row

,0,1,2,3,4,5,6,7,8,9,...,9413,9414,9415,9416,9417,9418,9419,9420,9421,9422
2103,2103.0,920.0,7185.0,6877.0,5827.0,8008.0,6892.0,1842.0,2675.0,8026.0,...,7799.0,6211.0,306.0,3481.0,1907.0,6162.0,7739.0,3796.0,9019.0,8322.0


In [234]:
rated_id = ratings.loc[ratings['user_id'] == 123]

print(rated_id['imdbID'])

,Unnamed: 0,user_id,imdbID,rating
2700,2770,123,107688.0,5.0
20972,21562,123,319061.0,4.0
51933,53382,123,118715.0,5.0
69225,71148,123,119822.0,3.5
91942,94427,123,338135.0,5.0
...,...,...,...,...
742419,765960,123,248845.0,5.0
745967,769632,123,112281.0,1.5
768099,792589,123,108052.0,5.0
779240,804157,123,181875.0,5.0


In [164]:
rated = np.empty((len(rated_id), 1))

In [238]:
i = 0
for imdb in rated_id['imdbID']:
        #print(imdb)
    rated[i] = omdb.loc[(omdb['imdbID'] == imdb)].index
    i = i+1
        
print(rated.shape)

(68, 1)
[[ 531.]
 [6431.]
 [1537.]
 [1575.]
 [6258.]
 [2396.]
 [6997.]
 [ 564.]
 [ 980.]
 [1106.]
 [ 394.]
 [  31.]
 [ 884.]
 [2410.]
 [1884.]
 [  24.]
 [5240.]
 [7487.]
 [3421.]
 [8564.]
 [3804.]
 [1885.]
 [5447.]
 [1021.]
 [1214.]
 [1137.]
 [1152.]
 [2417.]
 [5325.]
 [ 589.]
 [1238.]
 [1900.]
 [5426.]
 [1505.]
 [ 329.]
 [2762.]
 [ 785.]
 [1688.]
 [ 271.]
 [ 488.]
 [ 106.]
 [1338.]
 [3930.]
 [3739.]
 [7920.]
 [6257.]
 [1089.]
 [8182.]
 [7806.]
 [2439.]
 [ 124.]
 [ 572.]
 [ 331.]
 [3532.]
 [4508.]
 [ 839.]
 [2470.]
 [8411.]
 [2557.]
 [ 413.]
 [1102.]
 [ 138.]
 [3963.]
 [4182.]
 [  18.]
 [ 507.]
 [3486.]
 [ 285.]]


In [172]:
movie_row.columns = movie_row.loc[2103]
movie_row.drop(2103, inplace=True)
movie_row

2103,2103.0,920.0,7185.0,6877.0,5827.0,8008.0,6892.0,1842.0,2675.0,8026.0,...,7799.0,6211.0,306.0,3481.0,1907.0,6162.0,7739.0,3796.0,9019.0,8322.0


In [173]:
movie_row.loc[1,:] = range(0,len(indices))
movie_row

2103,2103.0,920.0,7185.0,6877.0,5827.0,8008.0,6892.0,1842.0,2675.0,8026.0,...,7799.0,6211.0,306.0,3481.0,1907.0,6162.0,7739.0,3796.0,9019.0,8322.0
1,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,9413.0,9414.0,9415.0,9416.0,9417.0,9418.0,9419.0,9420.0,9421.0,9422.0


In [252]:
rated_df = pd.DataFrame(rated)
rated_df

,0
0,531.0
1,6431.0
2,1537.0
3,1575.0
4,6258.0
...,...
63,4182.0
64,18.0
65,507.0
66,3486.0


In [253]:
# Position der bewerteten Filme innerhalb der Nachbarschaft
position = movie_row[list(rated_df[0].astype(int))]
position = position.transpose()
position.columns = ['position']
position ['Index'] = position.index
position = position.reset_index(drop = True)
position


,position,Index
0,3085.0,531.0
1,7128.0,6431.0
2,8900.0,1537.0
3,7169.0,1575.0
4,4638.0,6258.0
...,...,...
63,7303.0,4182.0
64,6279.0,18.0
65,9402.0,507.0
66,5639.0,3486.0


In [254]:
movie_row_distance = distances_df.loc[index]
movie_row_distance

,0,1,2,3,4,5,6,7,8,9,...,9413,9414,9415,9416,9417,9418,9419,9420,9421,9422
2103,5.960464e-08,1.002632,1.106629,1.125169,1.187902,1.192592,1.194088,1.199758,1.20744,1.222145,...,19.430275,20.955046,21.287977,23.20395,23.387875,24.813493,25.789218,26.777983,28.927177,50.43208


In [255]:
user_distances = movie_row_distance[list(position['position'].astype(int))].transpose()
user_distances.columns = ['distance']
user_distances =user_distances.reset_index(drop = True)
user_distances

,distance
0,3.808429
1,5.451673
2,7.447682
3,5.477868
4,4.374814
...,...
63,5.555586
64,5.013525
65,16.202719
66,4.742512


In [256]:
neighbors = pd.concat([position, user_distances], axis=1, join = "inner")
neighbors

,position,Index,distance
0,3085.0,531.0,3.808429
1,7128.0,6431.0,5.451673
2,8900.0,1537.0,7.447682
3,7169.0,1575.0,5.477868
4,4638.0,6258.0,4.374814
...,...,...,...
63,7303.0,4182.0,5.555586
64,6279.0,18.0,5.013525
65,9402.0,507.0,16.202719
66,5639.0,3486.0,4.742512


In [257]:
neighbors = neighbors.sort_values("distance")

In [264]:
rated_id['Index']= rated
neighbors = neighbors.merge(rated_id[["rating","Index"]], on="Index", how = "outer")
neighbors

<ipython-input-264-2ede6e933456>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rated_id['Index']= rated


,position,Index,distance,rating
0,530.0,1214.0,2.300173,2.5
1,1075.0,589.0,2.765940,5.0
2,1711.0,2410.0,3.185557,0.5
3,2911.0,1152.0,3.738719,3.5
4,3043.0,394.0,3.797212,4.5
...,...,...,...,...
63,9372.0,1106.0,13.946603,5.0
64,9384.0,785.0,14.635383,5.0
65,9387.0,1102.0,14.800018,5.0
66,9395.0,285.0,15.549070,5.0


In [269]:
# Hier weitermachen 
value1 = (1-(neighbors['distance']/neighbors['distance'].sum()))*neighbors['rating']
value1

0     2.486412
1     4.967320
2     0.496236
3     3.469079
4     4.459622
        ...   
63    4.835220
64    4.827082
65    4.825137
66    4.816287
67    4.808564
Length: 68, dtype: float64

## HIER WEITERARBEITEN ##

In [236]:
omdb

,imdbID,Year,Runtime,imdbRating,imdbVotes,Rotten Tomatoes,Metacritic,Series,Released_month,PG_Rating,Available_languages,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,114709.0,1995.0,81.0,8.3,852896.0,10.0,9.5,0.0,11.0,0.0,1.0,0.0,3.0,0.0,0.0
1,113497.0,1995.0,104.0,7.0,294340.0,5.4,3.9,0.0,12.0,1.0,2.0,0.0,0.0,0.0,0.0
2,107050.0,1993.0,103.0,7.0,41401.0,6.3,5.3,0.0,12.0,2.0,1.0,0.0,0.0,0.0,0.0
3,114885.0,1995.0,124.0,5.9,9222.0,5.6,5.9,0.0,12.0,3.0,1.0,0.0,0.0,0.0,0.0
4,113041.0,1995.0,106.0,6.0,33005.0,4.8,4.9,0.0,12.0,1.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9418,953318.0,2007.0,93.0,7.4,18103.0,6.8,5.9,0.0,9.0,0.0,2.0,0.0,0.0,0.0,0.0
9419,960731.0,2008.0,99.0,6.0,86128.0,2.6,3.3,0.0,12.0,1.0,1.0,0.0,0.0,0.0,0.0
9420,25464.0,1934.0,93.0,7.2,3084.0,8.0,5.9,0.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0
9421,1024715.0,2008.0,92.0,6.4,31058.0,5.5,4.7,0.0,9.0,3.0,1.0,0.0,0.0,0.0,0.0


In [43]:
def predict_movie_rating(userID, imdbID):
        
    #imdbID den Indices zuordnen
    index = omdb.loc[(omdb['imdbID'] == imdbID)].index
    #indices aller nachbarn von nächsten Nachbarn zum letzten nachbarn
    movie_row = indices[[index]]
    print(movie_row)
    #Entstehendes Dataframe für Filme Filtern, die der User bewertet hat (ersten 20 nehmen)
    #imdbIDs die der User bewertet hat
    rated_id = ratings.loc[ratings['user_id'] == userID]
    print(rated_id['imdbID'])
    rated = np.empty((len(rated_id), 1))
    
    #Zuordnung der imbdID  zu dem Indexen der von dem User bewerteten filme
    i = 0
    for imdb in rated_id['imdbID']:
        #print(imdb)
        rated[i] = omdb.loc[(omdb['imdbID'] == imdb)].index
        i = i+1
        
    print(rated.shape)
    
    # Suche ersten 20 bewertete Nachbarfilme in Nachbarschaft
  
    movie_frame = pd.DataFrame(movie_row)

   
    movie_frame.columns = movie_frame.iloc[0]
    movie_frame.loc[0] = np.arange(0,len(indices))
    print(movie_frame)
    
    #print(len(neighbor_idx))
    #print(neighbor_idx)
    #auf basis des abstandes dieser Nachbarn ein rating vorhersagen
    

In [52]:
predict_movie_rating(123, 145653.0)

[[ 124.  142. 2849. ... 3796. 9019. 8322.]]
2700      107688.0
20972     319061.0
51933     118715.0
69225     119822.0
91942     338135.0
            ...   
742419    248845.0
745967    112281.0
768099    108052.0
779240    181875.0
780940    110912.0
Name: imdbID, Length: 68, dtype: float64
(68, 1)
0  0.0     1.0     2.0     3.0     4.0     5.0     6.0     7.0     8.0     \
0     0.0     1.0     2.0     3.0     4.0     5.0     6.0     7.0     8.0   

0  9.0     ...  9413.0  9414.0  9415.0  9416.0  9417.0  9418.0  9419.0  \
0     9.0  ...  9413.0  9414.0  9415.0  9416.0  9417.0  9418.0  9419.0   

0  9420.0  9421.0  9422.0  
0  9420.0  9421.0  9422.0  

[1 rows x 9423 columns]


<ipython-input-51-cfb43a9141da>:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  movie_row = indices[[index]]


# Bis hier hin aktuell

In [53]:
#function that computes the k-nearest neighbors of each movie based on the features
for i in omdb['imdbID'].unique():
    try:
        distances , indices = knn.kneighbors(omdb.loc[omdb['imdbID'] == i], n_neighbors=10)
    except ValueError:
        print('Ohoh, ValueError')

In [ ]:
user_ratings['imdbID'] = user_ratings['imdbID'].astype(float)

In [ ]:
user_ratings

In [ ]:
user_ratings.dtypes

In [ ]:
# function that computes a rating based on the neighbors
def compute_rating(neighbors, distances):
    # TODO: Function schreiben, die statt dem mean() der Nachbarn die Ratings auf Basis des Abstandes gewichtet
    # Weiter entfernte Filme werden geringer gewichtet nähere Filme werden stärker gewichtet
    
    pred = np.mean(neighbors['rating'])
    
    return pred

In [ ]:
# function that computes a rating based on the neighbors
def compute_rating(neighbors, distances):
    # TODO: Function schreiben, die statt dem mean() der Nachbarn die Ratings auf Basis des Abstandes gewichtet
    # Weiter entfernte Filme werden geringer gewichtet nähere Filme werden stärker gewichtet
    
    pred = np.mean(neighbors['rating'])
    
    return pred

In [ ]:
## function that predicts the rating of a movie from its imdbID and its 20 nearest neighbors
# calculate similarity not based on user ratings? (only movie features)

def predict_movie_rating(imdbID, userID, user_data):
    # get 20 (or x) nearest neighbors that were rated by userID
    # TODO: Use function from kNN.ipynb to find optimal k for algorithm 
    k_neighbors = 10    
    
    # Select all ratings given by User #userID
    ratings = user_data.loc[user_data['user_id'] == userID]
    
    # Get real rating -> remove this in the end -> currently done for validation
    real_ratings = ratings.loc[(ratings['imdbID'] == imdbID)]
    
    #remove real rating
    ratings = ratings[ratings['imdbID'] != imdbID] 
     
    
    if (ratings.to_numpy().size>0):   
        
        # TODO:
        # Convert to Sparse Matrix Format
        #ratings_as_csr = csr_matrix(user_ratings)
        
        # Set algorithm and params 
        # TODO adapt neighbors based on number of given ratings by user_id ?
        knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=k_neighbors, n_jobs=-1)

        # Training
        print('---- Training ConBF-kNN-Algorithm ----')
        print('user_id: '+str(userID))
        print('imdbID: '+str(imdbID))
        knn.fit(ratings)

        
        #Prediction -> get x nearest neighbors of imdbID
        distances , indices = knn.kneighbors(user_data.loc[user_data['imdbID'] == imdbID], n_neighbors=k_neighbors)
        
       
        # Collect neighbors by indices
        for i in range(0,indices.shape[1]):
            neighbors = ratings.iloc[[indices[0,i]]]
            
        # compute rating of movie(imbdID) based on the rating of the 20 nearest neighbors
        pred = compute_rating(neighbors, distances)
        
        #Generate Output for Understandability
        print('Predicted Rating for '+str(imdbID)+': '+str(pred))
        print('Real Rating of '+str(imdbID)+' was: '+ str(real_ratings['rating'].values[0]))
        
        # return rating prediction and real rating
        return pred , real_ratings['rating'].values[0]
        
    else:
         return "User has not rated other movies. Check input"
    

In [ ]:
# Larger Test:
# TODO Refactoring needed #UglyPythonCode

def test_predict_mr(no_test_samples):
    # Sampling #no_test_samples of random samples from dataset for testing
    test_set = user_ratings.sample(n=no_test_samples)
    
    predictions = pd.DataFrame(columns=['Prediction'])
    reals = pd.DataFrame(columns=['Real_Rating'])
    
    # Iterate over test-set and generate predicitons for it
    # TODO get rid of ugly for-loop
    for row in test_set.itertuples():
        imdbID = row.imdbID
        userID = row.user_id
        pred , real = predict_movie_rating(imdbID, userID, user_ratings)
        predictions.loc[row.index] = pred
        reals.loc[row.index] = real
    
    rmse = mean_squared_error(reals['Real_Rating'], predictions['Prediction'], squared=False)
    print('RMSE: '+str(rmse))

In [ ]:
n = 80
test_predict_mr(n)